# Car Sales Analysis (Canada)
Mini-project to analyse [Canada Cars Sales Figures (2019-2021)](https://www.kaggle.com/mohamedhanyyy/canada-cars-sales-figures-20192021); a Kaggle dataset as 2 Sep 2021 (Version 6).

The purpose is to practice and apply Python (Pandas) & SQL (SQLite) knowledge.

Would love to know what you guys think, so if you have any feedback, please feel free to add it in the "Issues" section above!

## Setup
### Importing libraries

In [1]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine

### Creating SQL engine

In [2]:
engine = create_engine('sqlite://', echo=False)

### Importing CSV & converting to SQL

In [3]:
car_sales_df = pd.read_csv('Canadasalesdata.csv')

car_sales_df.to_sql('car_sales', engine, if_exists='replace', index=False)

## Analysis
### Initial exploration

In [4]:
car_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Model       236 non-null    object
 1   Jan         236 non-null    int64 
 2   Feb         236 non-null    int64 
 3   Mar         236 non-null    int64 
 4   Apr         236 non-null    int64 
 5   May         236 non-null    int64 
 6   Jun         236 non-null    int64 
 7   Jul         236 non-null    int64 
 8   Aug         236 non-null    int64 
 9   Sep         236 non-null    int64 
 10  Oct         236 non-null    int64 
 11  Nov         236 non-null    int64 
 12  Dec         236 non-null    int64 
 13  Sumofsales  236 non-null    int64 
 14  Category    236 non-null    object
 15  Year        236 non-null    int64 
dtypes: int64(14), object(2)
memory usage: 29.6+ KB


In [5]:
pd.read_sql_query("""SELECT *
                     FROM car_sales
                     LIMIT 10""", engine)

,Model,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Sumofsales,Category,Year
0,Acura MDX,93,389,782,611,550,501,0,0,0,0,0,0,2926,MLS,2021
1,Audi Q7,210,210,236,303,292,303,0,0,0,0,0,0,1554,MLS,2021
2,Audi Q8,130,130,146,211,203,211,0,0,0,0,0,0,1031,MLS,2021
3,BMW X5,323,323,364,518,498,518,0,0,0,0,0,0,2544,MLS,2021
4,BMW X6,72,72,81,113,108,113,0,0,0,0,0,0,559,MLS,2021
5,Cadillac XT5,378,378,425,575,553,575,0,0,0,0,0,0,2884,MLS,2021
6,Cadillac XT6,146,146,164,175,168,175,0,0,0,0,0,0,974,MLS,2021
7,Genesis GV80,108,106,160,154,146,168,121,0,0,0,0,0,963,MLS,2021
8,Infiniti QX60,130,130,146,95,91,95,0,0,0,0,0,0,687,MLS,2021
9,Jaguar F0Pace,116,116,131,184,177,184,0,0,0,0,0,0,908,MLS,2021


**Row 7** is one `Model` where there is Jul-2021 numbers. It is important to find out which other rows (if any) there are that is the same as this may affect calculations involving `Sumofsales` for 2021.

### Cleaning

#### Missing values

In [6]:
car_sales_df.isnull().sum()

Model         0
Jan           0
Feb           0
Mar           0
Apr           0
May           0
Jun           0
Jul           0
Aug           0
Sep           0
Oct           0
Nov           0
Dec           0
Sumofsales    0
Category      0
Year          0
dtype: int64

#### Removing Jul 2021 to ensure consistency

In [7]:
pd.read_sql_query("""SELECT model, jul, sumofsales, year
                     FROM car_sales
                     WHERE year = 2021
                         AND jul != 0""", engine)

,Model,Jul,Sumofsales,Year
0,Genesis GV80,121,963,2021
1,Lexus GX,14,175,2021
2,Lexus RX,923,5352,2021
3,Lexus NX,822,4441,2021
4,Lexus UX,337,1862,2021
5,Lexus LX,34,297,2021
6,Toyota Land Cruiser,43,196,2021
7,Toyota Sequoia,31,270,2021
8,Kia Sorento,835,5971,2021
9,Kia Telluride,293,2824,2021


To ensure that calculations involving the Total Sales of 2021 thus far are correct, a new column will be created called `Sumofsales_adj`.

In [8]:
car_sales_df['Sumofsales_adj'] = car_sales_df.apply(lambda row: 
                                                    row['Sumofsales'] - row['Jul'] 
                                                    if row['Year'] == 2021
                                                    else row['Sumofsales'],
                                                    axis=1)

In [9]:
car_sales_df.to_sql('car_sales', engine, if_exists='replace', index=False)

In [10]:
car_sales_df.head(10)

,Model,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Sumofsales,Category,Year,Sumofsales_adj
0,Acura MDX,93,389,782,611,550,501,0,0,0,0,0,0,2926,MLS,2021,2926
1,Audi Q7,210,210,236,303,292,303,0,0,0,0,0,0,1554,MLS,2021,1554
2,Audi Q8,130,130,146,211,203,211,0,0,0,0,0,0,1031,MLS,2021,1031
3,BMW X5,323,323,364,518,498,518,0,0,0,0,0,0,2544,MLS,2021,2544
4,BMW X6,72,72,81,113,108,113,0,0,0,0,0,0,559,MLS,2021,559
5,Cadillac XT5,378,378,425,575,553,575,0,0,0,0,0,0,2884,MLS,2021,2884
6,Cadillac XT6,146,146,164,175,168,175,0,0,0,0,0,0,974,MLS,2021,974
7,Genesis GV80,108,106,160,154,146,168,121,0,0,0,0,0,963,MLS,2021,842
8,Infiniti QX60,130,130,146,95,91,95,0,0,0,0,0,0,687,MLS,2021,687
9,Jaguar F0Pace,116,116,131,184,177,184,0,0,0,0,0,0,908,MLS,2021,908


### Exploratory Data Analysis

#### Total cars sold in each year

In [21]:
pd.read_sql_query("""SELECT year, SUM(sumofsales_adj) 'total_sales'
                     FROM car_sales
                     GROUP BY 1
                     ORDER BY 1 DESC""", engine)

,Year,total_sales
0,2021,128041
1,2020,341736
2,2019,390825


#### Most sold model each year
NB: 2021 only includes sales up to 1 July.

In [12]:
pd.read_sql_query("""SELECT Year, Model, Category, MAX(Sumofsales_adj) Sales
                     FROM car_sales
                     GROUP BY 1
                     ORDER BY 1 DESC""", engine)

,Year,Model,Category,Sales
0,2021,Toyota Highlander,MS,10403
1,2020,Jeep Wrangler,MS,22707
2,2019,Jeep Wrangler,MS,23185


#### Least sold model each year
NB: 2021 only includes sales up to 1 July.

In [30]:
pd.read_sql_query("""SELECT Year, Model, Category, MIN(Sumofsales_adj) Sales
                     FROM car_sales
                     GROUP BY 1
                     ORDER BY 1 DESC""", engine)

,Year,Model,Category,Sales
0,2021,Jeep Grand Wagoneer,LS,4
1,2020,Volkswagen Touareg,MS,2
2,2019,Volkswagen Touareg,MS,26


#### Most sold model each year, by category
Although all three highest-sold models by year are `MS` (Mid-sized SUV), which model sold the most in each year, in each category?

In [13]:
pd.read_sql_query("""SELECT Year, Category, Model, MAX(Sumofsales_adj) Sales
                     FROM car_sales
                     GROUP BY 2, 1
                     ORDER BY 1 DESC, 4 DESC""", engine)

,Year,Category,Model,Sales
0,2021,MS,Toyota Highlander,10403
1,2021,SLS,Audi Q5,5702
2,2021,MLS,Lexus RX,4429
3,2021,LS,GMC Yukon,3874
4,2021,LLS,Cadillac Escalade,1694
5,2020,MS,Jeep Wrangler,22707
6,2020,MLS,Lexus RX,9228
7,2020,SLS,Audi Q5,8048
8,2020,LS,Ford Expedition,4408
9,2020,LLS,Mercedes0Benz GL/GLS0Class,2256


From this, it seems that, consistently over 2019 to 2021 thus far, the order of most to least sold cars by categories are:
1. MS
2. SLS
3. MLS
4. LS
5. LLS

#### Most sold make each year

Each of the model name has the first word (some with the second as well, e.g. `Alfa Romeo`). I can get the first word of every row in `Model` using:

In [17]:
car_sales_df['Make'] = car_sales_df['Model'].str.split(' ').str.get(0)

In [18]:
car_sales_df.to_sql('car_sales', engine, if_exists='replace', index=False)

In [19]:
car_sales_df.head(10)

,Model,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Sumofsales,Category,Year,Sumofsales_adj,Make
0,Acura MDX,93,389,782,611,550,501,0,0,0,0,0,0,2926,MLS,2021,2926,Acura
1,Audi Q7,210,210,236,303,292,303,0,0,0,0,0,0,1554,MLS,2021,1554,Audi
2,Audi Q8,130,130,146,211,203,211,0,0,0,0,0,0,1031,MLS,2021,1031,Audi
3,BMW X5,323,323,364,518,498,518,0,0,0,0,0,0,2544,MLS,2021,2544,BMW
4,BMW X6,72,72,81,113,108,113,0,0,0,0,0,0,559,MLS,2021,559,BMW
5,Cadillac XT5,378,378,425,575,553,575,0,0,0,0,0,0,2884,MLS,2021,2884,Cadillac
6,Cadillac XT6,146,146,164,175,168,175,0,0,0,0,0,0,974,MLS,2021,974,Cadillac
7,Genesis GV80,108,106,160,154,146,168,121,0,0,0,0,0,963,MLS,2021,842,Genesis
8,Infiniti QX60,130,130,146,95,91,95,0,0,0,0,0,0,687,MLS,2021,687,Infiniti
9,Jaguar F0Pace,116,116,131,184,177,184,0,0,0,0,0,0,908,MLS,2021,908,Jaguar


In [29]:
pd.read_sql_query("""WITH m AS(
                         SELECT Year, Make, SUM(Sumofsales_adj) S
                         FROM car_sales
                         GROUP BY 2, 1
                         ORDER BY 1 DESC)
                     SELECT Year, Make, MAX(s) Sales
                     FROM m
                     GROUP BY 1
                     ORDER BY 1 DESC""", engine)

,Year,Make,Sales
0,2021,Toyota,14903
1,2020,Jeep,39674
2,2019,Jeep,41844


## Conclusion
Thank you for reviewing my code! If you have any suggestions, please feel free to post it in the "Issues" tab in GitHub, or comment below in Kaggle!